# CSU EDA v0.2
- 2022-01-01 to present (starts on a saturday)
- DMA split 3 ways:
    - awareness_markets: Orlando, Jacksonville, Greenville, SC and Birmingham
    - Mobile (DMA)
    - all other DMA
- it's been decided to only use cohort view (KPI by Lead date)
- Media Data (set up by Mike for Geo): Bing, Google, Meta, TikTok, TTD, LinkedIn

In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib as mpl

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.precision', 2)
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [2]:
kpi_raw = pd.read_csv('../data/CSU_KPI_raw_2022-01-01_2025-01-15.csv')
tiktok_raw = pd.read_csv('../data/CSU_data_tiktok_raw_2024-01-15.csv')
meta_raw = pd.read_csv('../data/CSU_data_meta_raw_2024-01-15.csv')
google_raw = pd.read_csv('../data/CSU_data_google_raw_2024-01-15.csv')
bing_raw = pd.read_csv('../data/CSU_data_bing_raw_2024-01-15.csv')
ttd_raw = pd.read_csv('../data/CSU_data_TTD_raw_2024-01-15.csv')
linkedin_raw = pd.read_csv('../data/CSU_data_linkedin_raw_2024-01-15.csv')

In [3]:
list_awareness = ['GREENVLL-SPART-ASHEVLL-AND', 'GREENWOOD-GREENVILLE', 'JACKSONVILLE', 'ORLANDO-DAYTONA BCH-MELBRN', 'BIRMINGHAM (ANN AND TUSC)']

df_kpi = kpi_raw.copy()
df_kpi['leaddate'] = pd.to_datetime(df_kpi['leaddate']).dt.normalize()
df_kpi['appdate'] = pd.to_datetime(df_kpi['appdate']).dt.normalize()
df_kpi['enrolldate'] = pd.to_datetime(df_kpi['enrolldate']).dt.normalize()
df_kpi['dma'] = None
df_kpi.loc[df_kpi['dmaname'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_kpi.loc[df_kpi['dmaname'].str.contains('MOBILE-PENSACOLA', na=False), 'dma'] = 'mobile'
df_kpi.loc[df_kpi['dma'].isna(), 'dma'] = 'other'
df_kpi = df_kpi.loc[df_kpi['leaddate'] < pd.Timestamp.today()].drop(['dmaname'], axis=1) 
df_kpi['channel'] = None
df_kpi.loc[df_kpi['dw_vendor'] == 'ArcherOrg', 'channel'] = 'Organic'
df_kpi.loc[df_kpi['dw_channel'].str.contains('Referral'), 'channel'] = 'Referral'
df_kpi.loc[df_kpi['dw_channel'].str.contains('PPL'), 'channel'] = 'PPL'
df_kpi.loc[df_kpi['channel'].isna(), 'channel'] = 'ALL(No PPL,Referral)' #NO PPL BUT Everything else
df_kpi = df_kpi.loc[df_kpi.leaddate < '2025-01-10'] ##keeping with saturday-friday

# ### Making cohort KPI data frame where all KPI are by the lead date (drop organic being separated, roll into ALL with referral)
df_kpi_cohort = df_kpi.groupby(['leaddate', 'dma', 'channel'])[['n_lead', 'n_app', 'n_enroll']].agg('sum')
df_kpi_cohort = df_kpi_cohort.unstack(['dma', 'channel']).reorder_levels([1,2, 0], axis=1).sort_index(axis=1).fillna(0)

df_kpi_cohort_weekly = df_kpi_cohort.resample('W-Sat').sum()

df_kpi_cohort_monthly = df_kpi_cohort.resample('MS').sum()

C:\Users\twolf\AppData\Local\Temp\ipykernel_28008\481306469.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_kpi.loc[df_kpi['dmaname'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'


In [4]:
df_kpi_cohort_monthly.head()

dma                   awareness                                                \
channel    ALL(No PPL,Referral)                 Organic                   PPL   
                          n_app n_enroll n_lead   n_app n_enroll n_lead n_app   
leaddate                                                                        
2022-01-01                30.00    12.00 129.00    0.00     0.00   0.00  4.00   
2022-02-01                24.00    10.00 125.00    0.00     0.00   0.00  6.00   
2022-03-01                32.00    13.00 108.00    0.00     0.00   0.00  8.00   
2022-04-01                23.00     9.00 116.00    0.00     0.00   0.00  5.00   
2022-05-01                43.00    15.00 137.00    0.00     0.00   0.00  8.00   

dma                                  ...                other          \
channel                    Referral  ... ALL(No PPL,Referral) Organic   
           n_enroll n_lead    n_app  ...               n_lead   n_app   
leaddate                             ...                                
2022-01-01     1.00  64.00     0.00  ...             4,406.00    0.00   
2022-02-01     0.00  74.00     0.00  ...             3,477.00    0.00   
2022-03-01     2.00  79.00     0.00  ...             3,551.00    0.00   
2022-04-01     1.00  55.00     0.00  ...             3,479.00    0.00   
2022-05-01     1.00  43.00     0.00  ...             3,504.00    0.00   

dma                                                                           
channel                       PPL                   Referral                  
           n_enroll n_lead  n_app n_enroll   n_lead    n_app n_enroll n_lead  
leaddate                                                                      
2022-01-01     0.00   0.00 175.00    25.00 1,421.00     0.00     0.00   0.00  
2022-02-01     0.00   0.00 146.00    23.00 1,761.00     0.00     0.00   0.00  
2022-03-01     0.00   0.00 182.00    31.00 2,252.00     0.00     0.00   0.00  
2022-04-01     0.00   0.00 105.00    17.00 1,217.00     0.00     0.00   0.00  
2022-05-01     0.00   0.00 108.00    15.00 1,007.00     0.00     0.00   0.00  

[5 rows x 36 columns]

- so i have split Media it how i see fit:
    - search: brand, nonBrand
    - video: youtube
    - social: meta, tiktok, linkedIn
    - display: display, Discovery, archerOrganic (supposed to be but isn't in yet)
    - TTD: ctv/ott, digital radio, OOH

In [5]:
list_awareness = ['GREENVILLE-SPARTANBURG, SC-ASHEVILLE, NC-ANDERSON, SC', 'JACKSONVILLE, FL', 'GREENWOOD-GREENVILLE, MS', 'ORLANDO-DAYTONA BEACH-MELBOURNE, FL', 'BIRMINGHAM, AL']

df_goo = google_raw.copy()
df_goo['Date'] = pd.to_datetime(df_goo['Date'])
df_goo['Metro area'] = df_goo['Metro area'].str.upper()
df_goo['dma'] = None
df_goo.loc[df_goo['Metro area'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_goo.loc[df_goo['Metro area'].str.contains('MOBILE', na=False), 'dma'] = 'mobile'
df_goo.loc[df_goo['dma'].isna(), 'dma'] = 'other'
df_goo['channel'] = None
df_goo.loc[df_goo['Brand'] == 'Brand', 'channel'] = 'SearchBrand'
df_goo.loc[df_goo['Campaign name'].str.contains('YouTube'), 'channel'] = 'Video'
df_goo.loc[df_goo['Campaign name'].str.contains('Display'), 'channel'] = 'Display'
df_goo.loc[df_goo['channel'].isna(), 'channel'] = 'SearchNonBrand' ###not certain if this is what we can make nonBrand
df_goo = df_goo.rename(columns={'Cost':'spend', 'Date':'date'})
df_goo = df_goo[['date', 'dma', 'channel', 'spend']]

In [6]:
df_goo

,date,dma,channel,spend
0,2022-07-01,other,SearchNonBrand,43.53
1,2022-07-02,other,SearchNonBrand,4.64
2,2022-07-02,other,SearchNonBrand,9.23
3,2022-07-02,other,SearchNonBrand,29.98
4,2022-07-03,other,SearchNonBrand,84.27
...,...,...,...,...
2464861,2022-06-28,other,Display,0.00
2464862,2022-06-29,other,Display,0.00
2464863,2022-06-30,other,Display,0.00
2464864,2022-06-30,other,Display,0.00


In [7]:
list_awareness = ['GREENVILLE-SPARTANBURG, SC-ASHEVILLE, NC-ANDERSON, SC', 'JACKSONVILLE, FL', 'GREENWOOD-GREENVILLE, MS', 'ORLANDO-DAYTONA BEACH-MELBOURNE, FL', 'BIRMINGHAM']

df_bing = bing_raw.copy()
df_bing['Date'] = pd.to_datetime(df_bing['Date'])
df_bing['Visitor metro area'] = df_bing['Visitor metro area'].str.upper()
df_bing['dma'] = None
df_bing.loc[df_bing['Visitor metro area'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_bing.loc[df_bing['Visitor metro area'].str.contains('MOBILE', na=False), 'dma'] = 'mobile'
df_bing.loc[df_bing['dma'].isna(), 'dma'] = 'other'
df_bing['channel'] = None
df_bing.loc[df_bing['Brand'] == 'Brand', 'channel'] = 'SearchBrand'
df_bing.loc[df_bing['Brand'] == 'Non Brand', 'channel'] = 'SearchNonBrand'
df_bing = df_bing.rename(columns={'Cost':'spend', 'Date':'date'})
df_bing = df_bing.groupby(['date', 'dma', 'channel'])['spend'].agg('sum').reset_index()

In [8]:
df_bing

,date,dma,channel,spend
0,2022-01-14,awareness,SearchBrand,91.48
1,2022-01-14,awareness,SearchNonBrand,13.86
2,2022-01-14,mobile,SearchBrand,33.18
3,2022-01-14,mobile,SearchNonBrand,10.65
4,2022-01-14,other,SearchBrand,"1,206.78"
...,...,...,...,...
6493,2024-12-31,awareness,SearchNonBrand,161.03
6494,2024-12-31,mobile,SearchBrand,35.95
6495,2024-12-31,mobile,SearchNonBrand,56.30
6496,2024-12-31,other,SearchBrand,"1,098.97"


In [9]:
list_awareness = ['GREENVILLE - SPARTANBURG - ASHEVILLE - ANDERSON', 'JACKSONVILLE', 'GREENWOOD - GREENVILLE', 'ORLANDO - DAYTONA BEACH - MELBOURNE', 'BIRMINGHAM']

df_tiktok = tiktok_raw.copy()
df_tiktok['Date'] = pd.to_datetime(df_tiktok['Date'])
df_tiktok['DMA name'] = df_tiktok['DMA name'].str.upper()
df_tiktok['dma'] = None
df_tiktok.loc[df_tiktok['DMA name'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_tiktok.loc[df_tiktok['DMA name'].str.contains('MOBILE', na=False), 'dma'] = 'mobile'
df_tiktok.loc[df_tiktok['dma'].isna(), 'dma'] = 'other'
df_tiktok['channel'] = 'Social'
df_tiktok = df_tiktok.rename(columns={'Cost':'spend', 'Date':'date'})
df_tiktok = df_tiktok[['date', 'dma', 'channel', 'spend']]

In [10]:
df_tiktok

,date,dma,channel,spend
0,2024-02-03,other,Social,7.00
1,2024-02-21,other,Social,7.00
2,2024-02-24,other,Social,7.00
3,2024-02-05,other,Social,7.25
4,2024-04-15,other,Social,7.50
...,...,...,...,...
335089,2023-11-23,other,Social,1.81
335090,2023-11-30,other,Social,1.81
335091,2023-12-03,other,Social,1.81
335092,2023-12-17,other,Social,1.81


In [11]:
list_awareness = ['GREENVLL-SPART-ASHEVLL-AND', 'GREENWOOD-GREENVILLE', 'JACKSONVILLE', 'ORLANDO-DAYTONA BCH-MELBRN', 'BIRMINGHAM']

df_meta = meta_raw.copy()
df_meta['Date'] = pd.to_datetime(df_meta['Date'])
df_meta['DMA'] = df_meta['DMA'].str.upper()
df_meta['dma'] = None
df_meta.loc[df_meta['DMA'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_meta.loc[df_meta['DMA'].str.contains('MOBILE', na=False), 'dma'] = 'mobile'
df_meta.loc[df_meta['dma'].isna(), 'dma'] = 'other'
df_meta['channel'] = 'Social'
df_meta = df_meta.rename(columns={'Cost':'spend', 'Date':'date'})
df_meta = df_meta[['date', 'dma', 'channel', 'spend']]

In [12]:
df_meta

,date,dma,channel,spend
0,2022-07-01,other,Social,0.65
1,2022-07-01,other,Social,1.53
2,2022-07-01,other,Social,0.69
3,2022-07-01,awareness,Social,3.55
4,2022-07-02,other,Social,1.22
...,...,...,...,...
1733750,2024-08-25,other,Social,0.07
1733751,2024-10-23,other,Social,0.14
1733752,2024-12-29,other,Social,0.14
1733753,2024-11-17,other,Social,0.14


In [13]:
list_awareness = ['GREENVILLE-SPARTANBURG SC-ASHEVILLE NC-ANDERSON SC', 'GREENWOOD-GREENVILLE', 'JACKSONVILLE', 'ORLANDO-DAYTONA BEACH-MELBOURNE', 'BIRMINGHAM']

df_ttd = ttd_raw.copy()
df_ttd['record_date'] = pd.to_datetime(df_ttd['record_date'])
df_ttd['metro'] = df_ttd['metro'].str.upper()
df_ttd['dma'] = None
df_ttd.loc[df_ttd['metro'].str.contains('|'.join(list_awareness), na=False), 'dma'] = 'awareness'
df_ttd.loc[df_ttd['dma'].isna(), 'dma'] = 'other'
df_ttd['channel'] = 'TTD'
df_ttd = df_ttd.rename(columns={'cost':'spend', 'record_date':'date'})
df_ttd = df_ttd[['date', 'dma', 'channel', 'spend']]

In [14]:
df_ttd

,date,dma,channel,spend
0,2024-06-24,awareness,TTD,"1,769.11"
1,2024-06-25,awareness,TTD,"1,269.76"
2,2024-06-26,awareness,TTD,"1,326.08"
3,2024-06-27,awareness,TTD,"1,357.42"
4,2024-06-28,awareness,TTD,"1,414.66"
...,...,...,...,...
10423,2024-09-12,awareness,TTD,0.00
10424,2024-09-13,awareness,TTD,0.00
10425,2024-09-14,awareness,TTD,0.00
10426,2024-09-15,awareness,TTD,0.00


In [15]:
 ### LinkedIn has no geo splitting from platform
df_li = linkedin_raw.copy()
df_li = df_li.rename(columns={'record_date':'date', 'cost':'spend'})
df_li['date'] = pd.to_datetime(df_li['date'])
df_li = df_li.groupby('date')['spend'].agg('sum').reset_index()
df_li['channel'] = 'Social'
df_li['dma'] = 'unknown'

In [16]:
df_li

,date,spend,channel,dma
0,2023-10-26,248.97,Social,unknown
1,2023-10-27,251.49,Social,unknown
2,2023-10-28,251.25,Social,unknown
3,2023-10-29,250.15,Social,unknown
4,2023-10-30,250.13,Social,unknown
...,...,...,...,...
443,2025-01-11,743.54,Social,unknown
444,2025-01-12,620.89,Social,unknown
445,2025-01-13,888.96,Social,unknown
446,2025-01-14,"1,227.14",Social,unknown


In [17]:
df_spend = pd.concat([df_bing, df_goo, df_meta, df_tiktok, df_ttd, df_li])
df_spend = df_spend.groupby(['date', 'dma', 'channel'])['spend'].agg('sum')
df_spend_daily = df_spend.reset_index().copy()

df_spend_weekly = df_spend.unstack(['dma', 'channel']).sort_index(axis=1).fillna(0)
df_spend_weekly = df_spend_weekly.resample('W-Sat').sum()

df_spend_monthly = df_spend.unstack(['dma', 'channel']).sort_index(axis=1).fillna(0)
df_spend_monthly = df_spend_monthly.resample('MS').sum()

In [25]:
today = pd.Timestamp.today().date()
today = pd.to_datetime(today).strftime('%Y-%m-%d')

### saving different df's to a single sheet
with pd.ExcelWriter(f'../data/csu_final_data_{today}.xlsx') as writer:
    df_kpi_cohort.to_excel(writer, sheet_name='KPI_cohort_daily', index=True, merge_cells=True)
    df_kpi_cohort_weekly.to_excel(writer, sheet_name='KPI_cohort_weekly', index=True, merge_cells=True)
    df_kpi_cohort_monthly.to_excel(writer, sheet_name='KPI_cohort_monthly', index=True, merge_cells=True)
    df_spend.to_excel(writer, sheet_name='spend_daily', index=True, merge_cells=True)  
    df_spend_weekly.to_excel(writer, sheet_name='spend_weekly', index=True, merge_cells=True)
    df_spend_monthly.to_excel(writer, sheet_name='spend_monthly', index=True, merge_cells=True)              

# EDA

In [19]:
df_kpi[['n_lead', 'n_app', 'n_enroll']].sum()

n_lead      226548
n_app        46373
n_enroll     16703
dtype: int64

In [20]:
df_kpi.groupby('dma')[['n_lead', 'n_app', 'n_enroll']].sum()

,n_lead,n_app,n_enroll
dma,,,
awareness,7550,1655,615
mobile,4656,1432,590
other,214342,43286,15498
